# TD2 : Spectral methods

In [1]:
import numpy as np
import itertools
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

## Part 1

### Load graph

In [2]:
nodes = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
edges = np.array([(1, 2), (2, 3), (3, 4), (5, 6), (5, 7), (6,7), (8, 10), (9, 10)])

### Compute adjacency matrix

In [3]:

def get_adjacency(nodes, edges):
    adjacency_matrix = np.zeros(shape=(nodes.shape[0], nodes.shape[0]))
    for (i, j) in edges:
        adjacency_matrix[i - 1, j - 1] = 1
        adjacency_matrix[j - 1, i - 1] = 1
    return adjacency_matrix

adjacency_matrix = get_adjacency(nodes, edges)

### Compute degrees

In [4]:
degrees = np.diag(np.sum(adjacency_matrix, axis = 0))
degrees

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]])

### Compute Laplacian

In [5]:
L = degrees - adjacency_matrix
L

array([[ 1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  2., -1., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  2., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1., -1.,  2.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  2.]])

In [6]:
eigenvalues, eigenvectors = np.linalg.eigh(L)
eigenvalues, eigenvectors

(array([-9.74614466e-17,  0.00000000e+00,  9.99658224e-17,  5.85786438e-01,
         1.00000000e+00,  2.00000000e+00,  3.00000000e+00,  3.00000000e+00,
         3.00000000e+00,  3.41421356e+00]),
 array([[-5.00000000e-01, -0.00000000e+00,  0.00000000e+00,
          6.53281482e-01,  0.00000000e+00,  5.00000000e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -2.70598050e-01],
        [-5.00000000e-01, -0.00000000e+00,  0.00000000e+00,
          2.70598050e-01,  0.00000000e+00, -5.00000000e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          6.53281482e-01],
        [-5.00000000e-01, -0.00000000e+00,  0.00000000e+00,
         -2.70598050e-01,  0.00000000e+00, -5.00000000e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -6.53281482e-01],
        [-5.00000000e-01, -0.00000000e+00,  0.00000000e+00,
         -6.53281482e-01,  0.00000000e+00,  5.00000000e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
   

Effectively, we have 3 eigenvalues equal to zero. Now, let's find the 3 eigenvectors, corresponding to the kernel of L.

In [7]:
kernel = eigenvectors[:, :3]
kernel

array([[-0.5       , -0.        ,  0.        ],
       [-0.5       , -0.        ,  0.        ],
       [-0.5       , -0.        ,  0.        ],
       [-0.5       , -0.        ,  0.        ],
       [ 0.        ,  0.57735027,  0.        ],
       [ 0.        ,  0.57735027,  0.        ],
       [ 0.        ,  0.57735027,  0.        ],
       [ 0.        , -0.        , -0.57735027],
       [ 0.        , -0.        , -0.57735027],
       [ 0.        ,  0.        , -0.57735027]])

## Part 2

In [57]:
nodes = np.array([1, 2, 3, 4, 5, 6])
edges = np.array([(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (2, 5), (3, 6), (6, 1)])

### Compute ratio cut cost

In [58]:
import itertools

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

def generator_all_combinations(edges):
    splitted_edges_set = powerset(edges)
    for splitted_edges in splitted_edges_set:
        yield splitted_edges



splitted_edges_set = generator_all_combinations(edges)

In [59]:
def get_partition(nodes, edges, splitted_edges):
    """Return partition from splitted edges"""
    adjacency_matrix = get_adjacency(nodes, edges)
    for edge in splitted_edges:
        adjacency_matrix[edge[0] - 1, edge[1] - 1] = 0
        adjacency_matrix[edge[1] - 1, edge[0] - 1] = 0
    graph = csr_matrix(adjacency_matrix)
    _, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
    return labels

In [60]:
def ratiocut(nodes, edges, splitted_edges):
    """Return ratio cut for splitted edges"""
    partition = get_partition(nodes, edges, splitted_edges)
    ratio_cut = 0
    for i in range(0, np.max(partition) + 1):
        nodes_in_partition = np.where(partition == i)[0]
        nodes_not_in_partition = np.where(partition != i)[0]
        cut = np.sum(adjacency_matrix[nodes_in_partition, :][:, nodes_not_in_partition])
        volume = nodes_in_partition.shape[0]
        if volume != 0:
            ratio_cut += cut / volume
    return ratio_cut

In [61]:
optimal_cut = np.inf
optimal_partition = None
optimal_splitted_edges = None
for index, splitted_edges in enumerate(generator_all_combinations(edges)):
    # Check if the partition is valid, and if it is, calculate the ratio cut
    partition = get_partition(nodes, edges, splitted_edges)
    if np.max(partition) >= 1:
        ratio_cut = ratiocut(nodes, edges, splitted_edges)
        # Check if the ratio cut is better than the previous optimal
        if ratio_cut < optimal_cut:
            optimal_cut = ratio_cut
            optimal_partition = partition
            optimal_splitted_edges = splitted_edges
        print(f"Ratio cut for {splitted_edges} is {ratiocut(nodes, edges, splitted_edges)}")
print(f"Optimal cut is {optimal_cut} for partition {optimal_partition} and splitted edges {optimal_splitted_edges}")

Ratio cut for (array([1, 2]), array([6, 1])) is 2.4
Ratio cut for (array([3, 4]), array([4, 5])) is 2.4
Ratio cut for (array([1, 2]), array([2, 3]), array([2, 5])) is 3.6
Ratio cut for (array([1, 2]), array([2, 3]), array([6, 1])) is 2.4
Ratio cut for (array([1, 2]), array([3, 4]), array([4, 5])) is 2.4
Ratio cut for (array([1, 2]), array([3, 4]), array([6, 1])) is 2.4
Ratio cut for (array([1, 2]), array([4, 5]), array([6, 1])) is 2.4
Ratio cut for (array([1, 2]), array([5, 6]), array([3, 6])) is 2.25
Ratio cut for (array([1, 2]), array([5, 6]), array([6, 1])) is 2.4
Ratio cut for (array([1, 2]), array([2, 5]), array([6, 1])) is 2.4
Ratio cut for (array([1, 2]), array([3, 6]), array([6, 1])) is 2.4
Ratio cut for (array([2, 3]), array([3, 4]), array([4, 5])) is 2.4
Ratio cut for (array([2, 3]), array([3, 4]), array([3, 6])) is 3.6
Ratio cut for (array([2, 3]), array([4, 5]), array([3, 6])) is 2.25
Ratio cut for (array([2, 3]), array([2, 5]), array([6, 1])) is 2.25
Ratio cut for (array([

### 2nd smallest eigenvalue

In [62]:
adjacency_matrix = get_adjacency(nodes, edges)
adjacency_matrix

array([[0., 1., 0., 0., 0., 1.],
       [1., 0., 1., 0., 1., 0.],
       [0., 1., 0., 1., 0., 1.],
       [0., 0., 1., 0., 1., 0.],
       [0., 1., 0., 1., 0., 1.],
       [1., 0., 1., 0., 1., 0.]])

In [63]:
degrees = np.diag(np.sum(adjacency_matrix, axis = 0))
degrees

array([[2., 0., 0., 0., 0., 0.],
       [0., 3., 0., 0., 0., 0.],
       [0., 0., 3., 0., 0., 0.],
       [0., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 3., 0.],
       [0., 0., 0., 0., 0., 3.]])

In [64]:
L = degrees - adjacency_matrix
L

array([[ 2., -1.,  0.,  0.,  0., -1.],
       [-1.,  3., -1.,  0., -1.,  0.],
       [ 0., -1.,  3., -1.,  0., -1.],
       [ 0.,  0., -1.,  2., -1.,  0.],
       [ 0., -1.,  0., -1.,  3., -1.],
       [-1.,  0., -1.,  0., -1.,  3.]])

In [65]:
eigenvalues, eigenvectors = np.linalg.eigh(L)
eigenvalues, eigenvectors

(array([-4.13783009e-16,  1.43844719e+00,  3.00000000e+00,  3.00000000e+00,
         3.00000000e+00,  5.56155281e+00]),
 array([[-4.08248290e-01,  6.57192300e-01, -4.90568122e-02,
         -5.75262342e-01,  0.00000000e+00,  2.60956474e-01],
        [-4.08248290e-01,  1.84524092e-01, -6.28487297e-01,
          3.43318579e-01, -2.65460739e-01, -4.64705132e-01],
        [-4.08248290e-01, -1.84524092e-01, -2.39972322e-01,
          3.10187076e-01,  6.55385838e-01,  4.64705132e-01],
        [-4.08248290e-01, -6.57192300e-01, -4.90568122e-02,
         -5.75262342e-01, -7.87104167e-17, -2.60956474e-01],
        [-4.08248290e-01, -1.84524092e-01,  2.89029134e-01,
          2.65075266e-01, -6.55385838e-01,  4.64705132e-01],
        [-4.08248290e-01,  1.84524092e-01,  6.77544109e-01,
          2.31943763e-01,  2.65460739e-01, -4.64705132e-01]]))

In [66]:
kernel = eigenvectors[:, 1]
kernel

array([ 0.6571923 ,  0.18452409, -0.18452409, -0.6571923 , -0.18452409,
        0.18452409])